In [17]:
import os
import pandas as pd #판다스 불러오기
import numpy as np
base_path = "E:\\wordmo\\" #해당 폴더안의
fileList = os.listdir(base_path)[1:]

# 모든 문장을 하나의 데이터프레임으로 concat
all_Sentences = pd.DataFrame()

for file in fileList:
    file_name = base_path + file
    if file.endswith("csv"): #모든 csv 파일 불러오기
        
        df = pd.read_csv(file_name, encoding="utf-8", header=None, names=[0]) # utf-8 로 인코딩
        if file.endswith("word3.csv"): #end with word 3
            df[0] = df[0].astype('str')
            df[0] = df[0].apply(lambda x: x.split("-")[0] if x.endswith("갤러리") else x)
            
        all_Sentences = pd.concat([all_Sentences, df])
        
    else:
        all_Sentences = pd.concat([all_Sentences, pd.read_excel(file_name, encoding="utf-8", header=None, names=[0])])
    print(file_name, len(df), len(all_Sentences))

E:\wordmo\word2.csv 11 11
E:\wordmo\word3.csv 1521 1532


In [18]:
# 중복되는 단어들을 제거 : 1532 => 1379
all_Sentences.drop_duplicates(inplace=True)
print(len(all_Sentences))

1379


In [19]:
# 특수문자나 한자 등이
#날라가지 않을 경우에 대하여
import re
pattern = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]") # 공백을 제외하고  제거

# 규표현식 적용
def clear_word(word):
    word = re.sub(pattern, "", word) #각 단어에 대하여 
    return word

all_Sentences[0] = all_Sentences.astype('str')
all_Sentences[0] = all_Sentences[0].apply(lambda x: clear_word(x)) #apply

In [20]:
# index 순서를 재정비
all_Sentences.reset_index(inplace=True) # reset_index
all_Sentences.drop('index', axis=1, inplace=True)
all_Sentences = all_Sentences[1:]

In [21]:
# 각 문장을 jamo_split하여 저장
from JamoSplit import jamo_split, jamo_combine
all_Sentences[0] = all_Sentences[0].apply(lambda x: jamo_split(x))
all_Sentences[0] = all_Sentences[0].apply(lambda x: x.split(" "))

In [22]:
# 데이터 확인: 정상적으로 자모분리한 형태로 문장단위로 만들어짐
all_Sentences

,0
1,"[ㅁㅣ_ㅊㅣㄴ, ㄴㅗㅁㅇㅣㄴㄱㅏ_, ㅇㅒ_ㄴㅡㄴ]"
2,"[ㅅㅣㅂㅍㅏㄹㄴㅕㄴㄷㅗ_ㅇㅔㄴ, ㅎㅏㄱㄱㅛ_ㅇㅔ_, ㄱㅏ_ㅇㅛ_]"
3,"[ㅁㅜ_ㅊㅣㄴㄴㅗㅁㅇㅣ_, ㅈㅣ_ㄴㅏ_ㄱㅏ_ㄴㅔ_, ㄴㅣ_ㄴㅏㅁㅊㅣㄴ]"
4,"[ㅇㅏ_ㄴㅣ_, ㄱㅡ_ㄴㅑㅇ, ㄱㅡ_ㄹㅓㄴㄱㅓ_ㄹㅏ_ㄱㅗ_, ㅅㅐㅇㄱㅏㄱㅎㅐ_, ㅅ..."
5,"[ㄲㅓ_ㅈㅕ_, ㅈㅓ_ㄹㅣ_ㄱㅏ_, ㅇㅣ_ㄴㅗㅁㅇㅏ_]"
...,...
1374,"[ㅇㅕ_ㅅㅓㅇㅎㅏㄱ, ㄷㅐ_ㅎㅏㄱㅇㅝㄴ]"
1375,"[ㅇㅢ_ㄹㅚ_ㅇㅣㄴㅇㅡㄹ, ㄴㅐ_, ㄱㅏ_ㅈㅗㄱㄱㅏㅌㅇㅣ_, ㅁㅗ_ㅅㅣ_ㄱㅔㅆㅅㅡㅂ..."
1376,"[ㅇㅓ_ㄱㅡ_, ㄱㅓ_ㅅㅣㄹㅎㅘ_]"
1377,"[ㄷㅗㅇㅅㅣ_ㅎㅡㅇ, ㅂㅜㄴㄱㅣ_ㅈㅓㅁ]"


In [23]:
# dataframe -> list로 변환
sentence_list = list(all_Sentences[0])
len(sentence_list)

1378

In [24]:
# fasttext 인풋 완성됐음을 확인
sentence_list[:10]

[['ㅁㅣ_ㅊㅣㄴ', 'ㄴㅗㅁㅇㅣㄴㄱㅏ_', 'ㅇㅒ_ㄴㅡㄴ'],
 ['ㅅㅣㅂㅍㅏㄹㄴㅕㄴㄷㅗ_ㅇㅔㄴ', 'ㅎㅏㄱㄱㅛ_ㅇㅔ_', 'ㄱㅏ_ㅇㅛ_'],
 ['ㅁㅜ_ㅊㅣㄴㄴㅗㅁㅇㅣ_', 'ㅈㅣ_ㄴㅏ_ㄱㅏ_ㄴㅔ_', 'ㄴㅣ_ㄴㅏㅁㅊㅣㄴ'],
 ['ㅇㅏ_ㄴㅣ_', 'ㄱㅡ_ㄴㅑㅇ', 'ㄱㅡ_ㄹㅓㄴㄱㅓ_ㄹㅏ_ㄱㅗ_', 'ㅅㅐㅇㄱㅏㄱㅎㅐ_', 'ㅅㅐ_ㄲㅣ_ㅇㅑ_'],
 ['ㄲㅓ_ㅈㅕ_', 'ㅈㅓ_ㄹㅣ_ㄱㅏ_', 'ㅇㅣ_ㄴㅗㅁㅇㅏ_'],
 ['ㄴㅐ_ㅁㅏㄹㅈㅗㅁ', 'ㄷㅡㄹㅇㅓ_ㅂㅘ_', 'ㅅㅣㅂㄹㅕㄴㅇㅏ_', 'ㅈㅗㅁ'],
 ['ㅅㅣ_ㅇㅣ_ㅂㅏㄹ', 'ㅂㅕ_ㅇㅓㅇㅅㅣㄴㅇㅣ_', 'ㅈㅣ_ㅇㅣ_ㄹㅏㄹ'],
 ['ㅇㅡㅇ', 'ㅈㅣ_ㄹㅏㄹ'],
 ['ㅈㅗ_ㅇㅗㅈㄲㅏ_', 'ㅅㅔ_ㅇㅛ_'],
 ['ㅇㅡㅇ', 'ㅇㅏ_ㄴㅣ_ㅇㅑ_']]

In [25]:
# fasttext애  적용할 차례
from gensim.models import FastText
# size: 좌우 2단어 
# min_count: 최소 1번 등장한 단어들
# workers: 4전부!!
# sg: 성능이 좋다
# min_n max_n : n-gram단위인. 1글자 ~ 2글자 최대 6-gram
# epochs: 반복횟수 
model = FastText(sentence_list, vector_size=50, window=2, min_count=1, workers=4, sg=1, min_n=3, max_n=6, epochs=10)

In [26]:
# 유사한 형태의 단어가 출력되는 것을 확인
model.wv.most_similar(jamo_split("개새끼"))

[('ㄱㅔ_ㅅㅐ_ㄲㅣ_', 0.8398352265357971),
 ('ㄱㅐ_ㅅㅐ_', 0.7305082082748413),
 ('ㄱㅐ_ㅅㅐ_ㄲㅑ_', 0.7271333336830139),
 ('ㅈㅗㅈㅅㅐ_ㄲㅣ_', 0.7021294832229614),
 ('ㅎㅗ_ㄹㅗ_ㅅㅐ_ㄲㅣ_', 0.697624146938324),
 ('ㄱㅐ_ㅎㅜ_ㄹㅏ_ㅅㅐ_ㄲㅣ_', 0.6936761140823364),
 ('ㄱㅐ_ㄸㅐ_ㄲㅣ_', 0.6807937026023865),
 ('ㄱㅐ_ㅅㅙㅅ', 0.6679026484489441),
 ('ㄷㅗㄴㅅㅐ_ㄲㅣ_', 0.6532419323921204),
 ('ㄱㅡ_ㅈㅣ_ㅅㅐ_ㄲㅣ_', 0.6515879034996033)]

In [12]:
# gensim_festtext.model => embedding model 저장
model.save("./gensim_festtext.model")

In [15]:
#from gensim.models import FastText
model = FastText(sentence_list, vector_size=50, window=2, min_count=1, workers=4, sg=1, min_n=3, max_n=6, epochs=10)

In [1]:
# 50개 문장만샘플링 하여 시각화 진행
import random
random_sentence_list = random.sample(sentence_list, 10)
random_word_list = pd.DataFrame([[word, model[word]]for sentence in random_sentence_list for word in sentence])

SyntaxError: invalid syntax (2166687020.py, line 1)